![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder

import datetime
from datetime import datetime

import re

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, поэтому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
Перед обработкой NAN выносим информацию о наличии пропуска как отдельный признак 

### 1.1 Number of Reviews

In [ ]:
def fill_number_of_reviews(data):
    data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
    #Заменяем пропуски в Number of Reviews на среднее. Пробовала заменить на среднее по городу, точность ухудшилась
    #data['Number of Reviews'] = data['Number of Reviews'].fillna(round(data['Number of Reviews'].mean()))
    data['Number of Reviews'] = data['Number of Reviews'].fillna(round(data.groupby('City')['Number of Reviews'].transform('mean')))

In [ ]:
fill_number_of_reviews(data)

### 1.2 Price Range

In [ ]:
def fill_price_range(data):
    # Сохраняем информацию о пропусках
    data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
    # Находим моду колонки Price Range и заменяем пропуски на моду
    price_mode = data['Price Range'].value_counts().index[0]
    data['Price Range'] = data['Price Range'].fillna(price_mode)

In [ ]:
fill_price_range(data)

### 1.3 Cuisine Style

In [ ]:
def fill_cuisine_style(data):
    # Сохраняем информацию о пропусках
    data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
    #Заполняем пропуски значением Other
    data['Cuisine Style'] = data['Cuisine Style'].fillna("['Other']")

In [ ]:
fill_cuisine_style(data)

## 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

### 2.1 Price Range

In [ ]:
def price_range_encode(data):
    # Кодируем значения колонки Price Range числами
    le = LabelEncoder()
    le.fit(data['Price Range'])
    data['Price Range LE'] = le.transform(data['Price Range'])

In [ ]:
price_range_encode(data)

### 2.2 City

In [ ]:
def city_encode(data):
    # применим One-Hot Encoding
    data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [ ]:
#city_encode(data)

### 2.3 Cuisine Style

In [ ]:
def cuisine_style_encode(data): 
    # Составим список из топ-10 кухонь, представленных в датасете
    # Предварительно приведём значения к корректному списку
    data['Cuisine Style'] = data['Cuisine Style'].str.replace('[', '')
    data['Cuisine Style'] = data['Cuisine Style'].str.replace(']', '')
    data['Cuisine Style'] = data['Cuisine Style'].str.replace(' ', '')
    data['Cuisine Style'] = data['Cuisine Style'].str.split(',')
    
    # Формируем список, значение Other исключаем
    cuisines = list(data[data['Cuisine_Style_isNAN'] == 0].explode('Cuisine Style')['Cuisine Style'].value_counts()[0:10].index[0:10])
    
    # Добавляем dummy-переменные для ресторанов из топ-10
    for cuisine in cuisines:
        data[cuisine] = data['Cuisine Style'].apply(lambda x: 1 if cuisine in x else 0)

In [ ]:
#cuisine_style_encode(data)

## 3 EDA 
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.
Имеет смысл добавить параметр, равный Ranking/количество ресторанов в городе

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),annot=True)


Видим, что имеющиеся на текущий момент признаки слабо скоррелированны, за исключением Cuisine_Stile_isNaN и Price_Range_isNaN. Можем сделать вывод, что у ресторанов, у которых не указана кухня, часто не указан и ценовой сегмент.
На основе матрицы корреляции принимаю решение оставить все признаки.

## 4 Создание новых признаков

### 4.1 Дельта между датами отзывов

In [ ]:
def set_reviews_date_delta(data):
    # Определим дельту между отзывами
    pattern = re.compile('\d+[/]\d+[/]\d+')
    data['Reviews Date'] = data['Reviews'].apply(lambda x: pattern.findall(str(x)))
    data['Reviews Date'] = data['Reviews Date'].apply(lambda x: [pd.to_datetime(i).date() for i in x])
    data['Reviews Date Delta'] = data['Reviews Date'].apply(lambda x: (x[0] - x[len(x)-1]) if len(x)>0 else None)
    data['Reviews Date Delta'] = data['Reviews Date Delta'].dt.days
    
    # Заполним пропуски средним значением
    data['Reviews Date Delta'] = data['Reviews Date Delta'].fillna(round(data.groupby('City')['Reviews Date Delta'].transform('mean')))

In [ ]:
set_reviews_date_delta(data)

### 4.2 Численность населения города

In [ ]:
def set_city_population_dict(data):
    # Словарь с численностью населения городов
    city_population_dict = {'Paris': 2160928, 'Stockholm': 975551, 'London': 8982256, 'Berlin': 3644826, 'Munich': 1471508, 'Oporto': 214349,
       'Milan': 1399860, 'Bratislava': 437726, 'Vienna': 1897491, 'Rome': 2873494, 'Barcelona': 1636762, 'Madrid': 3223334,
       'Dublin': 544107, 'Brussels': 174383, 'Zurich': 402762, 'Warsaw': 1764615, 'Budapest': 1752286, 'Copenhagen': 794128,
       'Amsterdam': 873555, 'Lyon': 513275, 'Hamburg': 1841179, 'Lisbon': 505526, 'Prague': 1308632, 'Oslo': 634293,
       'Helsinki': 656229, 'Edinburgh': 482005, 'Geneva': 499480, 'Ljubljana': 295504, 'Athens': 3167000,
       'Luxembourg': 124528, 'Krakow': 766683}
    # Добавляем колонку с численностью населения в датафрейм
    data['Population'] = data.City.map(city_population_dict)

In [ ]:
set_city_population_dict(data)

### 4.3 Признак "Столица"

In [ ]:
def set_capital(data):
    # Словарь со справочником столиц
    city_capital_dict = {'Paris': 1, 'Stockholm': 1, 'London': 1, 'Berlin': 1, 'Munich': 0, 'Oporto': 0,
       'Milan': 0, 'Bratislava': 1, 'Vienna': 1, 'Rome': 1, 'Barcelona': 0, 'Madrid': 1,
       'Dublin': 1, 'Brussels': 1, 'Zurich': 0, 'Warsaw': 1, 'Budapest': 1, 'Copenhagen': 1,
       'Amsterdam': 1, 'Lyon': 0, 'Hamburg': 0, 'Lisbon': 1, 'Prague': 1, 'Oslo': 1,
       'Helsinki': 1, 'Edinburgh': 1, 'Geneva': 0, 'Ljubljana': 1, 'Athens': 1,
       'Luxembourg': 1, 'Krakow': 0}
    # Добавляем колонку с признаком "Столица"
    data['Capital'] = data.City.map(city_capital_dict)

In [ ]:
#set_capital(data)

### 4.4 Плотность ресторанов на душу населения

In [ ]:
def set_restaurant_density(data):
    city_restaurant_count = data['City'].value_counts()
    data['Restaurant_count'] = data.City.map(city_restaurant_count)
    data['Restaurant_density'] = data['Restaurant_count'] / data['Population']

In [ ]:
set_restaurant_density(data)

### 4.5 Индекс покупательной способности населения города

In [ ]:
def set_purchasing_index(data):
    # Загружаем датасет с информацией о стоимости жизни по городам и покупательной способности населения.
    # Удалим всё, кроме города и покупательной способности
    cost_of_living = pd.read_csv('/kaggle/input/cost-of-living-index-by-country/Cost_of_living_index.csv')
    cost_of_living['City'] = cost_of_living['City'].str.split(',')
    cost_of_living['City'] = cost_of_living['City'].apply(lambda x: x[0])
    cost_of_living = cost_of_living.drop(cost_of_living[cost_of_living['Rank'] == 291].index)
    cost_of_living = cost_of_living.drop(['Rank', 'Cost of Living Index', 'Rent Index', 'Cost of Living Plus Rent Index', 'Groceries Index', 'Restaurant Price Index'], axis = 1)
    cost_of_living['City'] = cost_of_living['City'].apply(lambda x: 'Oporto' if x == 'Porto' else x)
    cost_of_living['City'] = cost_of_living['City'].apply(lambda x: 'Krakow' if x == 'Krakow (Cracow)' else x)
    return data.merge(cost_of_living, on='City', how='left')

In [ ]:
data = set_purchasing_index(data)

### 4.6 Ранг с учётом количества ресторанов в городе

In [ ]:
def set_weighed_ranking(data):
    data['Weighed_Ranking'] = data['Ranking'] / data['Restaurant_count']

In [ ]:
#set_weighed_ranking(data)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски
    fill_number_of_reviews(df_output) # количество отзывов
    fill_price_range(df_output) # ценовая категория
    fill_cuisine_style(df_output) # тип кухни    
    
    
    # ################### 3. Encoding ############################################################## 
    price_range_encode(df_output) # ценовая категория
    city_encode(df_output) # город
    cuisine_style_encode(df_output) # тип кухни (берём только топ-10)
    
    
    # ################### 4. Feature Engineering ####################################################
    set_reviews_date_delta(df_output) # дельта между датами последних отзывов
    set_city_population_dict(df_output) # численноссть населения города
    set_capital(df_output) # признак "Столица"
    set_restaurant_density(df_output) # плотность ресторанов на душу населения
    df_output = set_purchasing_index(df_output) # индекс покупательной способности населения
    set_weighed_ranking(df_output) # ранг с учётом количества ресторанов в городе
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
X_test.info()

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)
y_pred = np.round(y_pred * 2) / 2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
for i in range(predict_submission.size):
        predict_submission[i]=np.round(predict_submission[i] * 2) / 2

In [ ]:
predict_submission

In [ ]:
sample_submission.info()

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
